In [1]:
load_model = 'googlenet_based_model2.pt'

data_dir = '../../../stanford_dogs_new/'

In [2]:
# import numpy as np

from IPython.display import Image

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torchvision.datasets import ImageFolder
from PIL import Image

from torch.autograd import Variable

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


## load the Googlenet based model

In [3]:
googlenet_loaded = models.googlenet(pretrained=True)

for param in googlenet_loaded.parameters():
    param.requires_grad = False

googlenet_loaded.fc = nn.Linear(in_features=1024, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
googlenet_loaded = googlenet_loaded.to(device)



googlenet_loaded.load_state_dict(torch.load('googlenet_based_model2.pt'))
googlenet_loaded.eval()

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [4]:
model = googlenet_loaded

In [5]:
# preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
])

path = '../../../stanford_dogs_new/test'

list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

# gets the breed name from subfolders name like 'n02099429-curly-coated_retriever'
def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

per_class_accuracy = dict()
total_accuracy = 0

for breed_dir in list_subfolders_with_paths:
    
    breed = rename(breed_dir.split('/')[-1])
    
    images = os.listdir(breed_dir)
    
    matched = False
    matches = 0
    total_images = len(images)

    for image in images:

        input_image = Image.open(breed_dir + '/' + image)


        input_tensor = preprocess(input_image)
        if torch.cuda.is_available():
            input_tensor = Variable(input_tensor.cuda())

        input_batch = input_tensor.unsqueeze(0)
        out = model(input_batch)

        probabilities = torch.nn.functional.softmax(out[0], dim=0)
        # print(probabilities)

        with open("../../stanford_dogs_breeds_classes_final.txt", "r") as f:
            categories = [s.strip() for s in f.readlines()]

        predicted_breeds = []
        top1_prob, top1_catid = torch.topk(probabilities, 1)
        for i in range(top1_prob.size(0)):
            # predicted_breeds.append([categories[top3_catid[i]], top3_prob[i].item()*100])
            predicted_breed = categories[top1_catid[i]]

        # list to be used directly by the application (predicted_breed, probability)
        # print("predicted_breeds are: \n", predicted_breed)
        if breed == predicted_breed:
            matched = True
            matches = matches + 1

        per_class_accuracy[breed] = matches / total_images * 100

per_class_accuracy

{'Pomeranian': 95.45454545454545,
 'basset': 77.77777777777779,
 'Japanese spaniel': 100.0,
 'Bouvier des Flandres': 53.333333333333336,
 'German shepherd': 68.75,
 'Brabancon griffon': 87.5,
 'Blenheim spaniel': 100.0,
 'kelpie': 81.25,
 'Shih Tzu': 81.81818181818183,
 'curly coated retriever': 87.5,
 'Lakeland terrier': 35.0,
 'Irish setter': 93.75,
 'German short haired pointer': 81.25,
 'flat coated retriever': 68.75,
 'Irish water spaniel': 73.33333333333333,
 'Dandie Dinmont': 88.88888888888889,
 'clumber': 93.33333333333333,
 'Yorkshire terrier': 76.47058823529412,
 'schipperke': 100.0,
 'briard': 68.75,
 'miniature poodle': 31.25,
 'chow': 95.0,
 'Saluki': 80.0,
 'boxer': 56.25,
 'Pekinese': 60.0,
 'komondor': 87.5,
 'Brittany spaniel': 81.25,
 'black and tan coonhound': 68.75,
 'Appenzeller': 50.0,
 'Eskimo dog': 13.333333333333334,
 'giant schnauzer': 81.25,
 'Old English sheepdog': 100.0,
 'Chihuahua': 75.0,
 'miniature schnauzer': 81.25,
 'redbone': 80.0,
 'wire haired fox 

In [6]:
total_accuracy = sum(per_class_accuracy.values())/120
total_accuracy

77.17787542265096

### Top 10 Breeds with least accuracy

In [7]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1])[:k]

[('Eskimo dog', 13.333333333333334),
 ('standard schnauzer', 25.0),
 ('miniature poodle', 31.25),
 ('collie', 31.25),
 ('Lakeland terrier', 35.0),
 ('Lhasa', 42.10526315789473),
 ('whippet', 42.10526315789473),
 ('Appenzeller', 50.0),
 ('Walker hound', 50.0),
 ('Bouvier des Flandres', 53.333333333333336)]

### Top 15 Breeds with most accuracy

In [8]:
k = 15
sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True)[:k]

[('Japanese spaniel', 100.0),
 ('Blenheim spaniel', 100.0),
 ('schipperke', 100.0),
 ('Old English sheepdog', 100.0),
 ('Samoyed', 100.0),
 ('bull mastiff', 100.0),
 ('Rottweiler', 100.0),
 ('Bernese mountain dog', 100.0),
 ('groenendael', 100.0),
 ('African hunting dog', 100.0),
 ('Bedlington terrier', 100.0),
 ('Airedale', 100.0),
 ('Afghan hound', 95.83333333333334),
 ('Pomeranian', 95.45454545454545),
 ('Leonberg', 95.23809523809523)]